# Data Clean Up 

In [46]:
# Import modules
import os
import pandas as pd

## Make library data set

- Summary statistics of library stuff
- Totals per month of cases, tests, and deaths
- Zip code map of stats 

In [47]:
# Set parameters
covid_path = "../Resources/Weekly_COVID-19_Cases__Tests__and_Deaths_by_ZIP_Code_pulled_272022.csv"
library_circulation_2019_path = "../Resources/Libraries_-_2019_Circulation_by_Location_pulled_272022.csv"
library_visitors_2019_path = "../Resources/Libraries_-_2019_Visitors_by_Location_pulled_272022.csv"
library_circulation_2020_path = "../Resources/Libraries_-_2020_Circulation_by_Location_pulled_272022.csv"
library_visitors_2020_path = "../Resources/Libraries_-_2020_Visitors_by_Location_pulled_272022.csv"
library_circulation_2021_path = "../Resources/Libraries_-_2021_Circulation_by_Location_pulled_272022.csv"
library_visitors_2021_path = "../Resources/Libraries_-_2021_Visitors_by_Location_pulled_272022.csv"

In [48]:
# Load datasets
library_circulation_2019_df = pd.read_csv(library_circulation_2019_path, dtype={'ZIP': object})
library_visitors_2019_df = pd.read_csv(library_visitors_2019_path, dtype={'ZIP': object})
library_circulation_2020_df = pd.read_csv(library_circulation_2020_path, dtype={'ZIP': object})
library_visitors_2020_df = pd.read_csv(library_visitors_2020_path, dtype={'ZIP': object})
library_circulation_2021_df = pd.read_csv(library_circulation_2021_path, dtype={'ZIP': object})
library_visitors_2021_df = pd.read_csv(library_visitors_2021_path, dtype={'ZIP': object})

In [49]:
# For library circulation we have data without zip code (online renewal, media download, etc.)
# We want to drop this data since it has no zip code associated with it
library_circulation_2019_df = library_circulation_2019_df.dropna(how='any')
library_circulation_2020_df = library_circulation_2020_df.dropna(how='any')
library_circulation_2021_df = library_circulation_2021_df.dropna(how='any')

In [50]:
# Change LOCATION to BRANCH for 2019 data to match 2020 and 2021 format
library_circulation_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})
library_visitors_2019_df = library_circulation_2019_df.rename(columns={"LOCATION": "BRANCH"})

# Change Location to LOCATION
library_circulation_2021_df = library_circulation_2021_df.rename(columns={"Location": "LOCATION"})

In [51]:
# Add type column
library_circulation_2019_df["TYPE"] = ["Circulation"] * len(library_circulation_2019_df)
library_visitors_2019_df["TYPE"] = ["Visitors"] * len(library_visitors_2019_df)
library_circulation_2020_df["TYPE"] = ["Circulation"] * len(library_circulation_2020_df)
library_visitors_2020_df["TYPE"] = ["Visitors"] * len(library_visitors_2020_df)
library_circulation_2021_df["TYPE"] = ["Circulation"] * len(library_circulation_2021_df)
library_visitors_2021_df["TYPE"] = ["Visitors"] * len(library_visitors_2021_df)

# Add year column
library_circulation_2019_df["YEAR"] = [2019] * len(library_circulation_2019_df)
library_visitors_2019_df["YEAR"] = [2019] * len(library_visitors_2019_df)
library_circulation_2020_df["YEAR"] = [2020] * len(library_circulation_2020_df)
library_visitors_2020_df["YEAR"] = [2020] * len(library_visitors_2020_df)
library_circulation_2021_df["YEAR"] = [2021] * len(library_circulation_2021_df)
library_visitors_2021_df["YEAR"] = [2021] * len(library_visitors_2021_df)

In [52]:
# Merge circulation and visitors together
merged_2019 = pd.merge(library_circulation_2019_df, library_visitors_2019_df, how = "outer")
merged_2020 = pd.merge(library_circulation_2020_df, library_visitors_2020_df, how = "outer")
merged_2021 = pd.merge(library_circulation_2021_df, library_visitors_2021_df, how = "outer")

# Merge all together
merged = pd.merge(merged_2019, merged_2020, how = "outer")
merged = pd.merge(merged, merged_2021, how = "outer")

# Remove erroneous columns
merged = merged.drop(['LOCATION'], axis = 1)

# Remove index column
merged = merged.set_index('BRANCH')
merged.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR
BRANCH,,,,,,,,,,,,,,,,,,
Albany Park,3401 W. Foster Ave.,Chicago,60625,8214,7614,8460,7414,7044,7970,9664,9069,7053,8012,6571,5924,93009,Circulation,2019
Altgeld,13281 S. Corliss Ave.,Chicago,60827,378,326,332,270,320,325,354,229,234,377,369,500,4014,Circulation,2019
Archer Heights,5055 S. Archer Ave.,Chicago,60632,5365,5019,5417,5349,4833,4627,5356,4793,4537,5263,4051,3415,58025,Circulation,2019
Austin,5615 W. Race Ave.,Chicago,60644,215,590,706,736,806,983,1134,1236,1193,1361,1286,1082,11328,Circulation,2019
Austin-Irving,6100 W. Irving Park Rd.,Chicago,60634,9678,9764,10845,10541,9763,11442,11634,11063,9379,9932,8567,6492,119100,Circulation,2019


## Make COVID data set

In [54]:
# Load data
covid_df = pd.read_csv(covid_path)

# Align zip code column name
covid_df = covid_df.rename(columns={"ZIP Code": "ZIP"})

# Designate month and year using Week Start 
covid_df['YEAR'] = pd.to_datetime(covid_df['Week Start']).dt.year
covid_df['Month'] = pd.to_datetime(covid_df['Week Start']).dt.month
covid_df.head()

,ZIP,Week Number,Week Start,Week End,Cases - Weekly,Cases - Cumulative,Case Rate - Weekly,Case Rate - Cumulative,Tests - Weekly,Tests - Cumulative,...,Percent Tested Positive - Cumulative,Deaths - Weekly,Deaths - Cumulative,Death Rate - Weekly,Death Rate - Cumulative,Population,Row ID,ZIP Code Location,YEAR,Month
0,60612,15,4/5/2020,4/11/2020,69.0,202.0,201.0,588.7,284.0,803,...,0.3,3,7,8.7,20.4,34311,60612-2020-15,POINT (-87.687011 41.88004),2020,4
1,60612,16,4/12/2020,4/18/2020,66.0,268.0,192.0,781.1,286.0,1089,...,0.3,3,10,8.7,29.1,34311,60612-2020-16,POINT (-87.687011 41.88004),2020,4
2,60604,4,1/23/2022,1/29/2022,8.0,322.0,1023.0,41176.5,246.0,11461,...,0.0,0,0,0.0,0.0,782,60604-2022-4,POINT (-87.629029 41.878153),2022,1
3,60615,24,6/7/2020,6/13/2020,12.0,377.0,29.0,907.1,538.0,3986,...,0.1,0,15,0.0,36.1,41563,60615-2020-24,POINT (-87.602725 41.801993),2020,6
4,60615,25,6/14/2020,6/20/2020,13.0,390.0,31.0,938.3,551.0,4537,...,0.1,1,16,2.4,38.5,41563,60615-2020-25,POINT (-87.602725 41.801993),2020,6


In [55]:
# Group and sum cases-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
cases_weekly = pd.DataFrame(covid_grouped['Cases - Weekly'].sum())
cases_weekly = cases_weekly.reset_index()
cases_weekly_formatted = cases_weekly[ cases_weekly['Month'] == 1]
cases_weekly_formatted = cases_weekly_formatted.drop(['Month'], axis = 1)
cases_weekly_formatted = cases_weekly_formatted.rename(columns={"Cases - Weekly": "JANUARY"})

month_conversion = {2: "FEBRUARY",
                    3: "MARCH",
                    4: "APRIL",
                    5: "MAY",
                    6: "JUNE",
                    7: "JULY",
                    8: "AUGUST",
                    9: "SEPTEMBER",
                    10: "OCTOBER",
                    11: "NOVEMBER",
                    12: "DECEMBER"}

for month in range(2,13):
    cases_weekly_subset = cases_weekly[ cases_weekly['Month'] == month]
    cases_weekly_subset = cases_weekly_subset.drop(['Month'], axis = 1)
    cases_weekly_formatted = pd.merge(cases_weekly_formatted, cases_weekly_subset, how = "left", on = ["YEAR", "ZIP"])
    cases_weekly_formatted = cases_weekly_formatted.rename(columns={"Cases - Weekly": month_conversion[month]})
    
cases_weekly_formatted['TYPE'] = ['Cases'] * len(cases_weekly_formatted)
cases_weekly_formatted['YTD'] = cases_weekly_formatted.iloc[:,2:14].sum(axis = 1)
cases_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
115,2022,60661,333.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cases,333.0
116,2022,60666,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cases,1.0
117,2022,60707,492.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cases,492.0
118,2022,60827,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cases,86.0
119,2022,Unknown,2275.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cases,2275.0


In [56]:
# Group and sum deaths-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
deaths_weekly = pd.DataFrame(covid_grouped['Deaths - Weekly'].sum())
deaths_weekly = deaths_weekly.reset_index()
deaths_weekly_formatted = deaths_weekly[ deaths_weekly['Month'] == 1]
deaths_weekly_formatted = deaths_weekly_formatted.drop(['Month'], axis = 1)
deaths_weekly_formatted = deaths_weekly_formatted.rename(columns={"Deaths - Weekly": "JANUARY"})

for month in range(2,13):
    deaths_weekly_subset = deaths_weekly[ deaths_weekly['Month'] == month]
    deaths_weekly_subset = deaths_weekly_subset.drop(['Month'], axis = 1)
    deaths_weekly_formatted = pd.merge(deaths_weekly_formatted, deaths_weekly_subset, how = "left", on = ["YEAR", "ZIP"])
    deaths_weekly_formatted = deaths_weekly_formatted.rename(columns={"Deaths - Weekly": month_conversion[month]})
    
deaths_weekly_formatted['TYPE'] = ['Deaths'] * len(deaths_weekly_formatted)
deaths_weekly_formatted['YTD'] = deaths_weekly_formatted.iloc[:,2:14].sum(axis = 1)
deaths_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
115,2022,60661,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deaths,0.0
116,2022,60666,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deaths,0.0
117,2022,60707,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deaths,2.0
118,2022,60827,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deaths,1.0
119,2022,Unknown,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Deaths,5.0


In [57]:
# Group and sum deaths-weekly for a given year, month, and zip code
covid_grouped = covid_df.groupby(['YEAR', 'Month', 'ZIP'])
tests_weekly = pd.DataFrame(covid_grouped['Tests - Weekly'].sum())
tests_weekly = tests_weekly.reset_index()
tests_weekly_formatted = tests_weekly[ tests_weekly['Month'] == 1]
tests_weekly_formatted = tests_weekly_formatted.drop(['Month'], axis = 1)
tests_weekly_formatted = tests_weekly_formatted.rename(columns={"Tests - Weekly": "JANUARY"})

for month in range(2,13):
    tests_weekly_subset = tests_weekly[ tests_weekly['Month'] == month]
    tests_weekly_subset = tests_weekly_subset.drop(['Month'], axis = 1)
    tests_weekly_formatted = pd.merge(tests_weekly_formatted, tests_weekly_subset, how = "left", on = ["YEAR", "ZIP"])
    tests_weekly_formatted = tests_weekly_formatted.rename(columns={"Tests - Weekly": month_conversion[month]})
    
tests_weekly_formatted['TYPE'] = ['Tests'] * len(tests_weekly_formatted)
tests_weekly_formatted['YTD'] = tests_weekly_formatted.iloc[:,2:14].sum(axis = 1)
tests_weekly_formatted.tail()

,YEAR,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,TYPE,YTD
115,2022,60661,3373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tests,3373.0
116,2022,60666,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tests,0.0
117,2022,60707,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tests,0.0
118,2022,60827,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tests,0.0
119,2022,Unknown,11489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tests,11489.0


In [58]:
# Merge library and covid data
merged = pd.merge(merged, cases_weekly_formatted, how = "outer")
merged = pd.merge(merged, deaths_weekly_formatted, how = "outer")
merged = pd.merge(merged, tests_weekly_formatted, how = "outer")
merged.tail()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR
840,NaN,NaN,60661,3373.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3373.0,Tests,2022
841,NaN,NaN,60666,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tests,2022
842,NaN,NaN,60707,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tests,2022
843,NaN,NaN,60827,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,Tests,2022
844,NaN,NaN,Unknown,11489.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11489.0,Tests,2022


In [108]:
# Merge ZIP code coordinates
merged = pd.merge(merged, covid_df[["ZIP", "ZIP Code Location"]].drop_duplicates(), how = "left")
lat = merged["ZIP Code Location"].str.split(expand = True)[1].str.replace("(", "", regex = False)
lng = merged["ZIP Code Location"].str.split(expand = True)[2].str.replace(")", "", regex = False)
merged["LAT"] = lat
merged["LNG"] = lng
merged = merged.drop(['ZIP Code Location'], axis = 1)
merged.head()

,ADDRESS,CITY,ZIP,JANUARY,FEBRUARY,MARCH,APRIL,MAY,JUNE,JULY,AUGUST,SEPTEMBER,OCTOBER,NOVEMBER,DECEMBER,YTD,TYPE,YEAR,LAT,LNG
0,3401 W. Foster Ave.,Chicago,60625,8214.0,7614.0,8460.0,7414.0,7044.0,7970.0,9664.0,9069.0,7053.0,8012.0,6571.0,5924.0,93009.0,Circulation,2019,-87.701816,41.971155
1,13281 S. Corliss Ave.,Chicago,60827,378.0,326.0,332.0,270.0,320.0,325.0,354.0,229.0,234.0,377.0,369.0,500.0,4014.0,Circulation,2019,-87.633087,41.650765
2,5055 S. Archer Ave.,Chicago,60632,5365.0,5019.0,5417.0,5349.0,4833.0,4627.0,5356.0,4793.0,4537.0,5263.0,4051.0,3415.0,58025.0,Circulation,2019,-87.711251,41.810038
3,5615 W. Race Ave.,Chicago,60644,215.0,590.0,706.0,736.0,806.0,983.0,1134.0,1236.0,1193.0,1361.0,1286.0,1082.0,11328.0,Circulation,2019,-87.756863,41.881113
4,6100 W. Irving Park Rd.,Chicago,60634,9678.0,9764.0,10845.0,10541.0,9763.0,11442.0,11634.0,11063.0,9379.0,9932.0,8567.0,6492.0,119100.0,Circulation,2019,-87.797373,41.944967


In [109]:
# Export data
merged.to_csv("../Data/merged_library_covid.csv")

## Make sidewalk cafe permit data